## Build a pipeline for real time detection 

In [1]:
from roboflow import Roboflow
from transformers import DetrImageProcessor
import os
import torchvision
from PIL import Image
import random
import cv2
import numpy as np
import supervision as sv
from torch.utils.data import DataLoader
from torchvision import transforms
import torch
import pytorch_lightning as pl
from transformers import DetrForObjectDetection
import torch

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
rf = Roboflow(api_key="MSd5xP0ur783AweIVpqQ")
project = rf.workspace("license-plate-sf8t4").project("license-plate-detection-g15hx-hofxb")
version = project.version(1)
dataset = version.download("coco")

loading Roboflow workspace...
loading Roboflow project...


In [3]:
image_processor = DetrImageProcessor.from_pretrained('facebook/detr-resnet-50')

In [4]:
annotation_file = "_annotations.coco.json"
train_dir = os.path.join(dataset.location, 'train')
val_dir = os.path.join(dataset.location, 'valid')
test_dir = os.path.join(dataset.location, 'test')

In [5]:
class CocoDetection(torchvision.datasets.CocoDetection):
  def __init__( self, image_directory_path: str, image_processor, train: bool =True):
    annotation_file_path = os.path.join(image_directory_path, annotation_file)
    super(CocoDetection, self).__init__(image_directory_path, annotation_file_path)
    self.image_processor = image_processor

  def __getitem__(self, idx):
    images, annotations = super(CocoDetection, self).__getitem__(idx)
    image_id = self.ids[idx]
    annotations = {'image_id': image_id, 'annotations': annotations}
    encoding = self.image_processor(images=images, annotations=annotations, return_tensors='pt')
    pixel_values = encoding['pixel_values'].squeeze()
    target = encoding['labels'][0]
    return pixel_values, target

In [6]:
train_data = CocoDetection(
    image_directory_path = train_dir,
    image_processor = image_processor,
    train=True)

val_data = CocoDetection(
    image_directory_path=val_dir,
    image_processor = image_processor,
    train=False
)

test_data = CocoDetection(
    image_directory_path=test_dir,
    image_processor=image_processor,
    train=False
)

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [7]:
cat = test_data.coco.cats
id_to_lables = {k: v['name'] for k, v in cat.items()}
box_annot = sv.BoxAnnotator()

In [8]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model1 = DetrForObjectDetection.from_pretrained('/Users/sudipkhadka/Desktop/Object_Detection/Research/trained_model')
model1.to(DEVICE)

DetrForObjectDetection(
  (model): DetrModel(
    (backbone): DetrConvModel(
      (conv_encoder): DetrConvEncoder(
        (model): FeatureListNet(
          (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (bn1): DetrFrozenBatchNorm2d()
          (act1): ReLU(inplace=True)
          (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
          (layer1): Sequential(
            (0): Bottleneck(
              (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn1): DetrFrozenBatchNorm2d()
              (act1): ReLU(inplace=True)
              (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn2): DetrFrozenBatchNorm2d()
              (drop_block): Identity()
              (act2): ReLU(inplace=True)
              (aa): Identity()
              (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

In [10]:
cap = cv2.VideoCapture(1)

if not cap.isOpened():
    print('Error: Could not access camera')
    exit()
print("Searching for License Plates........ Press 'q' to exit")
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print('Error: Could not read frame')
            break
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        with torch.no_grad():
            input = image_processor(images=img, return_tensors='pt').to(DEVICE)
            outputs = model1(**input)

            CONFIDENCE_THRESHOLD =  0.5
            target_size = torch.tensor([img.shape[:2]]).to(DEVICE)
            results = image_processor.post_process_object_detection(outputs=outputs, threshold=CONFIDENCE_THRESHOLD, target_sizes=target_size)[0]

            detections = sv.Detections.from_transformers(transformers_results=results)

            labels = [f"{id_to_lables[class_id]} {confidence: .2f}" for _, confidence, class_id, _ in detections]
            frame = box_annot.annotate(scene=img.copy(), detections=detections, labels=labels)

            cv2.imshow('License Plate Detection', cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break   
finally:
    cap.release()
    cv2.destroyAllWindows()
    print('Camera Released and all windows closed')

Error: Could not access camera
Searching for License Plates........ Press 'q' to exit
Error: Could not read frame
Camera Released and all windows closed


OpenCV: out device of bound (0-0): 1
OpenCV: camera failed to properly initialize!


: 